In [1]:
import os
import sys
sys.dont_write_bytecode = True
os.environ['NUMBA_CACHE_DIR'] = os.path.join(os.getcwd(), 'numba_cache')
from matplotlib.colors import LogNorm
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

import pywt
BASE_PATH=os.getcwd()
OUTPUT_DIR = os.path.join(BASE_PATH, 'Images')
CSV_FILE_FULL_PATH = os.path.join(BASE_PATH, 'csv_files/train.csv') 
"""""
F_LOW = 100.0  
F_HIGH = 1500.0 
FILTER_ORDER = 4 
"""
#Filtry jednak nie potrzebne
"""""
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y_filtered = filtfilt(b, a, data)
    return y_filtered
"""
#Wizualizacje
def extract_features_for_csv(y):
    coeffs = pywt.wavedec(y, 'db4', level=5)
    cechy=[]
    for c in coeffs:
        mav=np.mean(np.abs(c))
        cechy.append(mav)
    return cechy
def plot_scalogram(y,fs,title_prefix,duration=5):
        time_limit = int(fs * duration)
        if len(y) > time_limit:
            y = y[:time_limit]
        scales = np.arange(1, 128)   
        coeffs, freqs = pywt.cwt(y, scales, 'morl', sampling_period=1/fs)
        power = np.abs(coeffs)
        plt.figure(figsize=(10, 6))
        time_duration = len(y) / fs
        plt.imshow(power, 
               extent=[0, time_duration, 1, 128],
               aspect='auto', 
               cmap='jet', 
               norm=LogNorm(vmin=power.min() + 1e-6, vmax=power.max()),
               origin='lower')          
        plt.colorbar(label='Energia')
        plt.title('Skalogram Falkowy (Morlet)')
        plt.xlabel('Czas (s)')
        plt.ylabel('Skala (częstotliwość)')
        save_path_fft = os.path.join(OUTPUT_DIR, f'{title_prefix}_skalogram.png')
        plt.savefig(save_path_fft) 
        plt.close()

        
def analyze_and_plot_signal(y, fs, title_prefix, filename):
    
    # ANALIZA CZĘSTOTLIWOŚCIOWA (FFT)
    N = len(y)
    Y = np.fft.fft(y)
    Y_abs = np.abs(Y) / N
    f = np.fft.fftfreq(N, 1/fs)
    jednostronne_index = np.where(f >= 0)
    f_jednostronne = f[jednostronne_index]
    Y_jednostronne = Y_abs[jednostronne_index]
    Y_jednostronne[1:] = 2 * Y_jednostronne[1:]

    # Wizualizacja Widma Amplitudowego (FFT)
    plt.figure(figsize=(10, 5))
    plt.plot(f_jednostronne, Y_jednostronne)
    plt.title(f'{title_prefix}: Widmo Amplitudowe (FFT)')
    plt.xlabel('Częstotliwość (Hz)')
    plt.ylabel('Amplituda (Normalizowana)')
    plt.xlim(0, fs / 2)
    plt.grid(True)
    plt.tight_layout()
    save_path_fft = os.path.join(OUTPUT_DIR, f'{title_prefix}_FFT.png')
    plt.savefig(save_path_fft)
    plt.close()

    #. WIZUALIZACJA SPEKTROGRAMÓW 
    S = librosa.stft(y)
    S_dB = librosa.amplitude_to_db(np.abs(S), ref=np.max)
    
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(S_dB, sr=fs, x_axis='time', y_axis='hz', cmap='viridis')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f'{title_prefix}: Spektrogram ')
    plt.ylim(0, 2000) 
    plt.tight_layout()
    save_path_fft = os.path.join(OUTPUT_DIR, f'{title_prefix}_spektogram.png')
    plt.savefig(save_path_fft) 
    plt.close()

# Wczytanie CSV z pełnej ścieżki
try:
    df_train = pd.read_csv(CSV_FILE_FULL_PATH) 
    print(f"Sukces: Wczytano {len(df_train)} rekordów z {CSV_FILE_FULL_PATH}")
except FileNotFoundError:
    print(f"BŁĄD KRYTYCZNY: Nie znaleziono pliku CSV: {CSV_FILE_FULL_PATH}. Sprawdź, czy plik jest w folderze PROJEKT.")
    exit()

# Wybór po jednym reprezentatywnym pliku z każdej klasy
try:
    lung_pathology = df_train[(df_train['label'] == 'Pathology') & (df_train['source'] == 'MIX')& (df_train['sound id'].str.replace(r'[^A-Za-z]', '', regex=True)=='L')].iloc[0]['filename']
    lung_normal = df_train[(df_train['label'] == 'Normal') & (df_train['source'] == 'LS')].iloc[0]['filename']
    heart_pathology = df_train[(df_train['label'] == 'Pathology') & (df_train['source'] == 'HS')].iloc[0]['filename']
    heart_normal = df_train[(df_train['label'] == 'Normal') & (df_train['source'] == 'HS')].iloc[0]['filename']
except IndexError:
    print("Błąd: Nie znaleziono wystarczającej liczby próbek 'Normal' lub 'Pathology' w train.csv.")
    exit()

files_to_process = {
    'lung_pathology': lung_pathology,
    'lung_normal': lung_normal,
    'heart_pathology': heart_pathology,
    'heart_normal': heart_normal
}


print("--- Rozpoczęcie przetwarzania sygnałów ---")

for label, filename in files_to_process.items():
    # Pełna ścieżka do pliku audio
    full_path = os.path.join(BASE_PATH, 'Sound', filename)
    print(f"\nPrzetwarzanie pliku: {label} ({full_path})")
    try:
        # Wczytanie sygnału audio
        y, fs = librosa.load(full_path, sr=16000)
        print(f"  Częstotliwość próbkowania (fs): {fs} Hz. Długość sygnału: {len(y)} próbek.")
        
        # Analiza i wizualizacja
        analyze_and_plot_signal(y, fs, label, filename)
        plot_scalogram(y,fs,label)
        print(f"  Analiza zakończona. Wykresy zapisane jako: {label}_*.png")

    except FileNotFoundError:
        print(f"  BŁĄD: Nie znaleziono pliku audio: {full_path}. Sprawdź, czy ścieżka w pliku CSV ({filename}) jest poprawna względem {BASE_PATH}!")
    except Exception as e:
        print(f"  Wystąpił nieznany błąd podczas przetwarzania: {e}")

print("\n--- Przetwarzanie ZAKOŃCZONE ---")

Sukces: Wczytano 292 rekordów z c:\Users\Karol\Desktop\Analiza_HLS-CMDS\csv_files/train.csv
--- Rozpoczęcie przetwarzania sygnałów ---

Przetwarzanie pliku: lung_pathology (c:\Users\Karol\Desktop\Analiza_HLS-CMDS\Sound\MIX/L0106.wav)
  Częstotliwość próbkowania (fs): 16000 Hz. Długość sygnału: 240000 próbek.
  Analiza zakończona. Wykresy zapisane jako: lung_pathology_*.png

Przetwarzanie pliku: lung_normal (c:\Users\Karol\Desktop\Analiza_HLS-CMDS\Sound\LS/M_N_LLA.wav)
  Częstotliwość próbkowania (fs): 16000 Hz. Długość sygnału: 240000 próbek.
  Analiza zakończona. Wykresy zapisane jako: lung_normal_*.png

Przetwarzanie pliku: heart_pathology (c:\Users\Karol\Desktop\Analiza_HLS-CMDS\Sound\HS/F_MSM_A.wav)
  Częstotliwość próbkowania (fs): 16000 Hz. Długość sygnału: 240000 próbek.
  Analiza zakończona. Wykresy zapisane jako: heart_pathology_*.png

Przetwarzanie pliku: heart_normal (c:\Users\Karol\Desktop\Analiza_HLS-CMDS\Sound\HS/F_N_RC.wav)
  Częstotliwość próbkowania (fs): 16000 Hz. Dłu